### Imports

In [1]:
### Standard imports
import os
import pickle
import random
import datetime
import numpy as np
import pandas as pd
import tensorflow as tf

from glob import glob
from os.path import join, isfile
from random import shuffle
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import LeakyReLU, ReLU
from tensorflow.keras.constraints import max_norm
from tensorflow.keras.callbacks import History
from tqdm.notebook import tqdm

tf.random.set_seed(5)
random.seed(5)
np.random.seed(5)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
os.environ['KMP_DUPLICATE_LIB_OK']='True'

root = "/Users/pstetz/Desktop/confidential/.project"

### Model

<div hidden>
def _load_model():
    lr = LeakyReLU(alpha=0.05); lr.__name__ = 'leaky_relu'
    relu = ReLU(); relu.__name__ = "relu"
    activation = relu
    use_bias = False
    def layer_a(dim):
        return layers.Dense(dim, activation=activation, use_bias=use_bias)
    
    info_input = keras.Input(shape=(130,), name="info")
    prev_input = keras.Input(shape=(9, 9, 9, 2), name="prev")
    next_input = keras.Input(shape=(9, 9, 9, 2), name="next")
    prev_s, next_s = [prev_input], [next_input]
    for i, layer in enumerate([
        layers.Conv3D(8, (2, 2, 2), use_bias=False),
        layers.MaxPool3D(),
        layers.Conv3D(8, (2, 2, 2), use_bias=False),
        layers.Conv3D(8, (2, 2, 2), use_bias=False),
        layers.Flatten()
    ]):
        prev_s.append(layer(prev_s[-1]))
        next_s.append(layer(next_s[-1]))
    info_s = [info_input]
#     for dim in (130, 130, 130):
#         info_s.append(layer_a(dim)(info_s[-1]))

    x_0 = layers.concatenate([prev_s[-1], next_s[-1], info_s[-1]])
    x_s = [x_0]
#     for dim in (146, 146, 146, 146, 128, 64, 32):
    for dim in (258, 258, 128, 128, 64, 32):
        x_s.append(layer_a(dim)(x_s[-1]))
    
    bold_signal = layers.Dense(1, name="bold_signal")(x_s[-1])
    model = keras.Model(inputs=[prev_input, next_input, info_input], outputs=[bold_signal])
    learning_rate = 1e-4
    model.compile(optimizer=keras.optimizers.SGD(lr=learning_rate, momentum=8e-2, decay=learning_rate/30),
      loss={"bold_signal": "mse"},
      loss_weights=[1.])
    return model
    
def _load_model(with_lgbm = False):
    lr = LeakyReLU(alpha=0.05); lr.__name__ = 'leaky_relu'
    relu = ReLU(); relu.__name__ = "relu"
#     activation = lr
    
    info_input = keras.Input(shape=(130 + int(with_lgbm),), name="info")
    prev_input = keras.Input(shape=(9, 9, 9, 2), name="prev")
    next_input = keras.Input(shape=(9, 9, 9, 2), name="next")
    prev_s, next_s = [prev_input], [next_input]
    for i, layer in enumerate([
        layers.Conv3D(32, (2, 2, 2), use_bias=False, activation="elu"),
        layers.Dropout(0.25),
        layers.Conv3D(32, (2, 2, 2), use_bias=False, activation="elu"),
        layers.MaxPooling3D(pool_size=(2, 2, 2)),
        layers.Flatten()
    ]):
        prev_s.append(layer(prev_s[-1]))
        next_s.append(layer(next_s[-1]))
    info_s = [info_input]
    x_0 = layers.concatenate([prev_s[-1], next_s[-1], info_s[-1]])
    x_s = [x_0]
    init_shape = x_0.shape[1]
    for layer in (
        layers.Reshape((init_shape, 1)),
        layers.Conv1D(256, 256, strides=init_shape, activation='elu'),
        layers.BatchNormalization(),
        layers.Reshape((256, 1)),
        layers.Dropout(0.15),
        layers.Conv1D(256, 256, strides=init_shape, activation='elu'),
        layers.BatchNormalization(),
        layers.Reshape((256, 1)),
        layers.Dropout(0.15),
        layers.Conv1D(128, 256, activation='elu'),
        layers.BatchNormalization(),
        layers.Reshape((128, 1)),
        layers.Conv1D(64, 128, activation='elu'),
        layers.BatchNormalization(),
        layers.Reshape((64, 1)),
        layers.Dropout(0.25),
        layers.Conv1D(32, 64, activation='elu'),
        layers.BatchNormalization(),
        layers.Dropout(0.25),
        layers.Reshape((32, 1)),
        layers.AveragePooling1D(2),
        layers.Flatten(),
    ):
        x_s.append(layer(x_s[-1]))
    
    bold_signal = layers.Dense(1, name="bold_signal")(x_s[-1])
    model = keras.Model(inputs=[prev_input, next_input, info_input], outputs=[bold_signal])
    learning_rate = 1e-3
    model.compile(
        optimizer=keras.optimizers.SGD(lr=learning_rate, momentum=8e-1, decay=learning_rate/30),
        loss={"bold_signal": "mse"}, loss_weights=[1.]
    )
    return model

if "model" in locals(): del model
model = _load_model()
</div>

In [2]:
def _load_model():
    lr = LeakyReLU(alpha=0.05); lr.__name__ = 'leaky_relu'
    relu = ReLU(); relu.__name__ = "relu"
    activation = lr
    use_bias = False
    def layer_a(dim):
        return layers.Dense(dim, activation=relu, use_bias=use_bias, kernel_constraint=max_norm(3))
    
    info_input = keras.Input(shape=(130,), name="info")
    prev_input = keras.Input(shape=(9, 9, 9, 2), name="prev")
    next_input = keras.Input(shape=(9, 9, 9, 2), name="next")
    prev_s, next_s = [prev_input], [next_input]
    for i, layer in enumerate([
        layers.Conv3D(4, (2, 2, 2), use_bias=False, kernel_constraint=max_norm(3)),
        layers.Conv3D(4, (2, 2, 2), use_bias=False, kernel_constraint=max_norm(3)),
        layers.Flatten(),
        layers.Dense(64, activation="elu", kernel_constraint=max_norm(3)),
    ]):
        prev_s.append(layer(prev_s[-1]))
        next_s.append(layer(next_s[-1]))
    info_s = [info_input]
    x_0 = layers.concatenate([prev_s[-1], next_s[-1], info_s[-1]])
    print(x_0.shape)
    x_s = [x_0]
    for dim in (258, 256, 256, 128, 128): #, 128, 128, 128, 64, 32):
        x_s.append(layer_a(dim)(x_s[-1]))
#         x_s.append(layers.BatchNormalization()(x_s[-1]))
#         x_s.append(layers.Dropout(0.5)(x_s[-1]))
    
    bold_signal = layers.Dense(1, name="bold_signal")(x_s[-1])
    model = keras.Model(inputs=[prev_input, next_input, info_input], outputs=[bold_signal])
    learning_rate = 5e-4
    model.compile(optimizer=keras.optimizers.SGD(lr=learning_rate, momentum=3e-2, decay=learning_rate/20),
      loss={"bold_signal": "mse"},
      loss_weights=[1.])
    return model

if "model" in locals(): del model
model = _load_model()

(None, 258)


### Setup

In [5]:
training_path = "/Users/pstetz/Desktop/confidential/.project/summary/consolidate"
log_dir = join(root, "logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = History()

batch_size = 32
num_epoches = 3

### Training

In [1]:
histories = list()
for _ in range(4):
    for batch_path in tqdm(glob(join(training_path, "*"))):
        batch = pickle.load(open(batch_path, "rb"))
        bold_signal = batch.pop("pred")
        model.fit(
            batch, bold_signal,
            epochs=num_epoches, batch_size=batch_size, verbose=True,
            callbacks=[tensorboard_callback, history],
            validation_split=0.2,
            shuffle=True # use when randomly selecting batches
        )
        histories.append(model.history.history)

### Save history

In [7]:
tmp = {"loss": [], "val_loss": []}
for h in histories:
    tmp["loss"].extend(h["loss"])
    tmp["val_loss"].extend(h["val_loss"])
    
pd.DataFrame(tmp).to_csv("/Users/pstetz/Desktop/confidential/.project/4_65_score.csv", index=False)

In [8]:
model.save("/Users/pstetz/Desktop/confidential/.project/run/4_65_score.h5")